In [7]:
import pandas as pd
import numpy as np

In [8]:
with open("smalldataset2.csv", 'w') as fle:
    with open("tweetid_userid_keyword_sentiments_emotions_United States.csv", 'r') as myfile:
        for x in range(5000000):
            fle.write(next(myfile))

In [16]:
df= pd.read_csv("smalldataset2.csv")

In [17]:
df.emotion.value_counts()

fear                   1478648
anger                  1297721
happiness              1209097
no specific emotion     804105
sadness                 210428
Name: emotion, dtype: int64

In [18]:
fear = df[df.emotion == 'fear'].sample(60000, random_state= 42)
anger = df[df.emotion == 'anger'].sample(60000, random_state= 42)
happiness = df[df.emotion == 'happiness'].sample(60000, random_state= 42)
sadness = df[df.emotion == 'sadness'].sample(60000, random_state= 42)
no_emotion = df[df.emotion == 'no specific emotion'].sample(60000, random_state= 42)

In [19]:
new_df = pd.concat([fear, anger, happiness, sadness, no_emotion])

In [20]:
new_df

,tweet_id,user_id,tweet_timestamp,keyword,valence_intensity,fear_intensity,anger_intensity,happiness_intensity,sadness_intensity,sentiment,emotion
420891,1234217430303506437,289118612,2020-03-01 12-42-20,wuhan,0.337,0.580,0.543,0.245,0.498,negative,fear
364939,1233576105153568770,90986776,2020-02-28 18-13-56,covid,0.464,0.508,0.280,0.217,0.398,negative,fear
1842971,1238967936800481287,4284288797,2020-03-14 16-19-09,corona,0.320,0.615,0.468,0.254,0.539,negative,fear
1148431,1237937409024610304,19975536,2020-03-11 20-04-12,wuhan,0.402,0.539,0.461,0.232,0.486,negative,fear
4158171,1243252528084631560,49097116,2020-03-26 12-04-35,covid,0.322,0.631,0.521,0.224,0.555,negative,fear
...,...,...,...,...,...,...,...,...,...,...,...
3680028,1242478392600965122,140888759,2020-03-24 08-48-27,covid,0.507,0.352,0.390,0.388,0.402,neutral or mixed,no specific emotion
2707470,1240480222010433538,1637759850,2020-03-18 20-28-26,corona,0.484,0.413,0.461,0.285,0.468,neutral or mixed,no specific emotion
3255699,1241586946427604992,3065121946,2020-03-21 21-46-10,covid,0.491,0.344,0.253,0.308,0.366,neutral or mixed,no specific emotion
243837,1232361218922381313,731394599952908288,2020-02-25 09-46-25,covid,0.481,0.586,0.376,0.327,0.427,neutral or mixed,no specific emotion


In [23]:
ids =new_df.tweet_id.to_list()

In [24]:
len(ids)

300000

In [25]:
import tweepy
from tqdm import tqdm

def lookup_tweets(tweetids, api):
    all_tweets = []
    total_tweets = len(tweetids)
    try:
        with tqdm(total= total_tweets // 100) as bar:
            for i in range((total_tweets // 100) + 1):

                #buffer for 100 tweet request limit
                end_loc = min((i + 1) * 100, total_tweets)
                
                #buffer for last list
                if len(tweetids[i * 100:end_loc]) != 0:
                    all_tweets.extend(
                        api.statuses_lookup(tweetids[i * 100:end_loc], tweet_mode='extended')
                    )
                bar.update(1)
            return all_tweets
    except tweepy.TweepError as e:
        print(e.args[0][0]['code'])

In [26]:
consumer_key = 'FtnVlVUM0Fsl1U53yNbzVe6MP'
consumer_secret = 'N7oRPDVHHmuBtU0wlgsKdElQznWjTI2wMUvFCt8nnXR6PJLVhw'
access_token = '1414623144057450497-rQgHhGRuAzofOOJHnpohuGR2BwTkoG'
access_secret = 'nI6E3YTrEVch6pUNMWl4n5qEIYuImUyl7Y8f6w0JyF2K6'

In [28]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

# do whatever it is to get por.TweetID - the list of all IDs to look up

results = lookup_tweets(ids, api)

 96%|███████████████████████████████████████████████████████████████████████████▏  | 2894/3000 [44:08<01:07,  1.56it/s]Rate limit reached. Sleeping for: 243
3001it [49:29,  1.01it/s]                                                                                              


In [29]:
results[3]._json

{'created_at': 'Thu Mar 05 02:20:23 +0000 2020',
 'id': 1235389666301362177,
 'id_str': '1235389666301362177',
 'full_text': "Working from home until 03/31 due to corona. I think I'm gonna go stir crazy and tweet 10x more",
 'truncated': False,
 'display_text_range': [0, 95],
 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': []},
 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 901036476,
  'id_str': '901036476',
  'name': 'Jake Blakeley',
  'screen_name': 'BlakeJakely',
  'location': 'Seattle, WA',
  'description': '🤫 Designing the future of AR @meta\n💥 OG Spark AR\n🖼️ Co-creator of 3D photos @facebook\n🐶 Malinios dad',
  'url': 'https://t.co/MSDQCHBI8i',
  'entities': {'url': {'urls': [{'url': 'https://t.co/MSDQCHBI8i',
      'expanded_url': 'ht

In [30]:
import json
temp = [status._json for status in results] #create list of dictionaries

In [31]:
tweetdf = pd.json_normalize(temp) #dictionaries to dataframe
tweetdf = tweetdf[['id','full_text']] #drop all but 

In [32]:
#merge dataframe on tweet_id
full = pd.merge(new_df, tweetdf, left_on='tweet_id', right_on='id', how='left').drop('id', axis=1)

In [33]:
#drop deleted tweets or tweets from banned individuals 
full.dropna(inplace= True)
full

,tweet_id,user_id,tweet_timestamp,keyword,valence_intensity,fear_intensity,anger_intensity,happiness_intensity,sadness_intensity,sentiment,emotion,full_text
0,1234217430303506437,289118612,2020-03-01 12-42-20,wuhan,0.337,0.580,0.543,0.245,0.498,negative,fear,"WE ALL KNOW GOVT's LIE, BUT...\n1,200 bodies a..."
1,1233576105153568770,90986776,2020-02-28 18-13-56,covid,0.464,0.508,0.280,0.217,0.398,negative,fear,Infection control guidance by CDC for COVID-19...
2,1238967936800481287,4284288797,2020-03-14 16-19-09,corona,0.320,0.615,0.468,0.254,0.539,negative,fear,I think I have the corona virus
3,1237937409024610304,19975536,2020-03-11 20-04-12,wuhan,0.402,0.539,0.461,0.232,0.486,negative,fear,@GOPChairwoman @realDonaldTrump On 31 Déc 2019...
5,1231866573289222145,962725727950049281,2020-02-24 01-00-52,covid,0.458,0.480,0.386,0.229,0.399,negative,fear,IMF warns Covid-19 threatens global economic r...
...,...,...,...,...,...,...,...,...,...,...,...,...
300436,1241077985085620225,1622270396,2020-03-20 12-03-44,corona,0.483,0.456,0.468,0.312,0.450,neutral or mixed,no specific emotion,Here is Azan the person saying( asalath u fil ...
300438,1238667846156054528,31908735,2020-03-13 20-26-42,covid,0.491,0.470,0.419,0.329,0.403,neutral or mixed,no specific emotion,"Major local COVID-19 news on Friday, March 13,..."
300439,1242478392600965122,140888759,2020-03-24 08-48-27,covid,0.507,0.352,0.390,0.388,0.402,neutral or mixed,no specific emotion,2nd Colorado city joins ‘stay-at-home’ order a...
300442,1232361218922381313,731394599952908288,2020-02-25 09-46-25,covid,0.481,0.586,0.376,0.327,0.427,neutral or mixed,no specific emotion,Coronavirus: China reports 508 new Covid-19 ca...


In [34]:
#Check for Balance
full.sentiment.value_counts()

negative            127779
neutral or mixed     49077
positive             49053
very negative        11738
very positive         1162
Name: sentiment, dtype: int64

In [36]:
full.emotion.value_counts()

happiness              50215
no specific emotion    49077
sadness                47877
fear                   46891
anger                  44749
Name: emotion, dtype: int64

In [38]:
full.to_csv("TwitterSentimentDataset2.csv", index = False, encoding= 'utf8')

In [42]:
df55= pd.read_csv("TwitterSentimentDataset2.csv", encoding= 'utf8')
df55.head(35)

,tweet_id,user_id,tweet_timestamp,keyword,valence_intensity,fear_intensity,anger_intensity,happiness_intensity,sadness_intensity,sentiment,emotion,full_text
0,1234217430303506437,289118612,2020-03-01 12-42-20,wuhan,0.337,0.580,0.543,0.245,0.498,negative,fear,"WE ALL KNOW GOVT's LIE, BUT...\n1,200 bodies a..."
1,1233576105153568770,90986776,2020-02-28 18-13-56,covid,0.464,0.508,0.280,0.217,0.398,negative,fear,Infection control guidance by CDC for COVID-19...
2,1238967936800481287,4284288797,2020-03-14 16-19-09,corona,0.320,0.615,0.468,0.254,0.539,negative,fear,I think I have the corona virus
3,1237937409024610304,19975536,2020-03-11 20-04-12,wuhan,0.402,0.539,0.461,0.232,0.486,negative,fear,@GOPChairwoman @realDonaldTrump On 31 Déc 2019...
4,1231866573289222145,962725727950049281,2020-02-24 01-00-52,covid,0.458,0.480,0.386,0.229,0.399,negative,fear,IMF warns Covid-19 threatens global economic r...
5,1237439471311708160,807440658684223488,2020-03-10 11-05-35,wuhan,0.375,0.460,0.383,0.207,0.365,negative,fear,@winyeemichelle Isn't it called the Wuhan Virus?
6,1238253010146463748,309783780,2020-03-12 16-58-17,covid,0.478,0.481,0.464,0.330,0.445,negative,fear,The St. Clair County Health Department adds th...
7,1237732163740741633,293876472,2020-03-11 06-28-38,corona,0.444,0.534,0.495,0.304,0.427,negative,fear,@VP @CDCgov @HHSGov \n@CNN OMG this man clay i...
8,1232136952561983488,835885102114750464,2020-02-24 18-55-16,corona,0.419,0.498,0.406,0.165,0.402,negative,fear,@EricBoehlert Is it wrong to hope the corona v...
9,1237445871316480008,16454070,2020-03-10 11-31-01,corona,0.337,0.604,0.512,0.245,0.502,negative,fear,Due to California declaring a state of emergen...


In [44]:
df55.head(35).full_text[30]

'we are not complaining about our job, we are SCARED! The country is under a #StateOfEmergency , but our company is doing NOTHING to protect us! Even the @CWAUnion is too scared to speak out for us... &amp; we’ve paid them Tens of Thousands of 💲 over our 30-40 year careers! 🥺#COVID'